In [1]:
headers = ({'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})

In [2]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.arabam.com/ikinci-el/otomobil/volkswagen-polo?take=50&page="
car_data = []

for page_number in range(1, 51):
    url = base_url + str(page_number)
    data = requests.get(url).text
    soup = BeautifulSoup(data, 'html.parser')
    listings = soup.find_all('tr', class_='listing-list-item should-hover bg-white')
    for listing in listings:
        model = listing.find('div', class_='listing-text-new').text
        year = listing.find('td', class_='listing-text').text.strip()
        kilometers = listing.find_all('td', class_='listing-text')[1].text.strip()
        color = listing.find_all('td', class_='listing-text')[2].text.strip()
        price = listing.find('span', class_='db no-wrap listing-price').text.strip()
            
        location = listing.find('span', class_='').text + " - " + listing.find_all('span', class_='')[1].text

        car_data.append([model, year, kilometers, color, price, location])

In [3]:
import pandas as pd
df = pd.DataFrame(car_data, columns=['Model', 'Year', 'Kilometers', 'Color', 'Price', 'Location'])
df

,Model,Year,Kilometers,Color,Price,Location
0,Volkswagen Polo 1.6 TDi Comfortline,2013,131.000,Siyah,655.000 TL,İstanbul - Avcılar
1,Volkswagen Polo 1.6 TDi Comfortline,2013,118.750,Gri,699.750 TL,Tokat - Niksar
2,Volkswagen Polo 1.4 TDi Comfortline,2014,137.000,Beyaz,749.000 TL,Ankara - Sincan
3,Volkswagen Polo 1.2 TSi Lounge,2016,144.200,Beyaz,676.750 TL,İstanbul - Maltepe
4,Volkswagen Polo 1.4 TDi Comfortline,2015,213.490,Beyaz,549.000 TL,İzmir - Gaziemir
...,...,...,...,...,...,...
2491,Volkswagen Polo 1.6,1997,222.500,Siyah,268.000 TL,Erzurum - Palandöken
2492,Volkswagen Polo 1.4 TDi Comfortline,2016,77.200,Beyaz,740.000 TL,Bursa - Nilüfer
2493,Volkswagen Polo 1.4 Comfortline,2010,87.000,Beyaz,650.000 TL,İzmir - Bornova
2494,Volkswagen Polo 1.6 TDi Comfortline,2013,252.000,Beyaz,610.000 TL,Antalya - Kepez


In [4]:
col = df["Model"].str.split(" ", 1, expand=True).rename({0:"Car Brand", 1:"CarModel"}, axis = 1)

In [5]:
df = df.join([col])

In [6]:
df = df.drop("Model", axis = 1)

In [7]:
df.isnull().sum()

Year          0
Kilometers    0
Color         0
Price         0
Location      0
Car Brand     0
CarModel      0
dtype: int64

In [8]:
col2 = df.CarModel.str.split(" ", 1, expand=True).rename({0:"Model", 1:"Engine"}, axis = 1)

In [9]:
df = df.join(col2)

In [10]:
df = df.drop('CarModel', axis = 1)

In [11]:
col3 = df.Price.str.split(" ", 1, expand = True).rename({0:"Price1", 1:"Currency"}, axis = 1)

In [12]:
df = df.join(col3)
df

,Year,Kilometers,Color,Price,Location,Car Brand,Model,Engine,Price1,Currency
0,2013,131.000,Siyah,655.000 TL,İstanbul - Avcılar,Volkswagen,Polo,1.6 TDi Comfortline,655.000,TL
1,2013,118.750,Gri,699.750 TL,Tokat - Niksar,Volkswagen,Polo,1.6 TDi Comfortline,699.750,TL
2,2014,137.000,Beyaz,749.000 TL,Ankara - Sincan,Volkswagen,Polo,1.4 TDi Comfortline,749.000,TL
3,2016,144.200,Beyaz,676.750 TL,İstanbul - Maltepe,Volkswagen,Polo,1.2 TSi Lounge,676.750,TL
4,2015,213.490,Beyaz,549.000 TL,İzmir - Gaziemir,Volkswagen,Polo,1.4 TDi Comfortline,549.000,TL
...,...,...,...,...,...,...,...,...,...,...
2491,1997,222.500,Siyah,268.000 TL,Erzurum - Palandöken,Volkswagen,Polo,1.6,268.000,TL
2492,2016,77.200,Beyaz,740.000 TL,Bursa - Nilüfer,Volkswagen,Polo,1.4 TDi Comfortline,740.000,TL
2493,2010,87.000,Beyaz,650.000 TL,İzmir - Bornova,Volkswagen,Polo,1.4 Comfortline,650.000,TL
2494,2013,252.000,Beyaz,610.000 TL,Antalya - Kepez,Volkswagen,Polo,1.6 TDi Comfortline,610.000,TL


In [13]:
if "Price" in df.columns:
    df = df.drop("Price", axis=1)
else:
    print("Column 'Price' not found in the dataset.")

print(df.head())

   Year Kilometers  Color            Location   Car Brand Model  \
0  2013    131.000  Siyah  İstanbul - Avcılar  Volkswagen  Polo   
1  2013    118.750    Gri      Tokat - Niksar  Volkswagen  Polo   
2  2014    137.000  Beyaz     Ankara - Sincan  Volkswagen  Polo   
3  2016    144.200  Beyaz  İstanbul - Maltepe  Volkswagen  Polo   
4  2015    213.490  Beyaz    İzmir - Gaziemir  Volkswagen  Polo   

                Engine   Price1 Currency  
0  1.6 TDi Comfortline  655.000       TL  
1  1.6 TDi Comfortline  699.750       TL  
2  1.4 TDi Comfortline  749.000       TL  
3       1.2 TSi Lounge  676.750       TL  
4  1.4 TDi Comfortline  549.000       TL  


In [14]:
col4 = df.Location.str.split(" ", 1, expand = True).rename({0:"City", 1:"District"}, axis = 1)
col4

,City,District
0,İstanbul,- Avcılar
1,Tokat,- Niksar
2,Ankara,- Sincan
3,İstanbul,- Maltepe
4,İzmir,- Gaziemir
...,...,...
2491,Erzurum,- Palandöken
2492,Bursa,- Nilüfer
2493,İzmir,- Bornova
2494,Antalya,- Kepez


In [15]:
df = df.join(col4)

In [16]:
df = df.drop("Location", axis = 1)

In [17]:
df['District'] = df['District'].str.replace("-", " ")

In [18]:
df.rename(columns = {"Price1":"Price"}, inplace = True)

In [19]:
df['Year'] = df['Year'].astype(int)

In [20]:
df['Kilometers'] = df['Kilometers'].str.replace('.', '')

C:\Users\user\AppData\Local\Temp\ipykernel_7388\2414687831.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Kilometers'] = df['Kilometers'].str.replace('.', '')


In [21]:
df['Kilometers'] = df['Kilometers'].str.replace('-', '').astype(int)

In [22]:
df['Kilometers'].astype(int)

0       131000
1       118750
2       137000
3       144200
4       213490
         ...  
2491    222500
2492     77200
2493     87000
2494    252000
2495     76000
Name: Kilometers, Length: 2496, dtype: int32

In [23]:
type(df['Kilometers'][0])

numpy.int32

In [24]:
df = df.drop("Currency", axis = 1)

In [25]:
col6 = df['Engine'].str.split(' ', n=1, expand=True).rename({0: 'Engine1', 1: 'Devami'}, axis=1)
col6

,Engine1,Devami
0,1.6,TDi Comfortline
1,1.6,TDi Comfortline
2,1.4,TDi Comfortline
3,1.2,TSi Lounge
4,1.4,TDi Comfortline
...,...,...
2491,1.6,None
2492,1.4,TDi Comfortline
2493,1.4,Comfortline
2494,1.6,TDi Comfortline


In [26]:
df = df.join(col6)

In [27]:
df['Engine1'] = df['Engine1'].astype(float)

In [28]:
df = df.drop(columns = 'Engine')

In [29]:
df.rename(columns = {"Engine1":"Engine"}, inplace = True)

In [30]:
df = df.drop(columns = 'Devami')

In [31]:
df['Price'] = df['Price'].str.replace('.', '').astype(int)

C:\Users\user\AppData\Local\Temp\ipykernel_7388\3128332942.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Price'] = df['Price'].str.replace('.', '').astype(int)


In [32]:
df.reset_index(drop = True, inplace = True)
df

,Year,Kilometers,Color,Car Brand,Model,Price,City,District,Engine
0,2013,131000,Siyah,Volkswagen,Polo,655000,İstanbul,Avcılar,1.6
1,2013,118750,Gri,Volkswagen,Polo,699750,Tokat,Niksar,1.6
2,2014,137000,Beyaz,Volkswagen,Polo,749000,Ankara,Sincan,1.4
3,2016,144200,Beyaz,Volkswagen,Polo,676750,İstanbul,Maltepe,1.2
4,2015,213490,Beyaz,Volkswagen,Polo,549000,İzmir,Gaziemir,1.4
...,...,...,...,...,...,...,...,...,...
2491,1997,222500,Siyah,Volkswagen,Polo,268000,Erzurum,Palandöken,1.6
2492,2016,77200,Beyaz,Volkswagen,Polo,740000,Bursa,Nilüfer,1.4
2493,2010,87000,Beyaz,Volkswagen,Polo,650000,İzmir,Bornova,1.4
2494,2013,252000,Beyaz,Volkswagen,Polo,610000,Antalya,Kepez,1.6


In [33]:
df.to_csv('car-sale-polo.csv', index = False)